## Part B: Transferring data to a relational database

Joining and filtering data can be easily done by SQL queries if we have data in a relational database. So, we are now transfering the csv files as three tables in a MySQL database

The following csv files were tansfered to a MySQL database 

1. 'distance_data_filtered.csv'
2. 'covid_data_filtered.csv'
3. 'population_data.csv'

We'll name the database table respectively as following:
1. `nycovidcase`
2. `nydistance`
3. `nypopulation`

### 1. Final checking of the content before sending to the database

In [2]:
import csv
n1=0
n2=0
n3=0

with open ('distance_data_filtered.csv', 'r') as f1:
    reader1 = csv.reader(f1)
    for row1 in reader1:
        print(row1)
        if n1>4:
            break
        n1+=1

        
with open ('covid_data_filtered.csv', 'r') as f2:
    reader2 = csv.reader(f2)
    for row2 in reader2:
        print(row2)
        if n2>4:
            break
        n2+=1

with open ('population_data.csv', 'r') as f3:
    reader3 = csv.reader(f3)
    for row3 in reader3:
        print(row3)
        if n3>4:
            break
        n3+=1

['fips', 'distance', 'fips2']
['36001', '16.23015283', '36093']
['36001', '22.6607026', '36039']
['36001', '23.61156982', '36095']
['36001', '24.86162199', '36083']
['36001', '29.44956155', '36021']
['date', 'county', 'state', 'fips', 'cases', 'deaths']
['2020-03-01', 'New York City', 'New York', '', '1', '0']
['2020-03-02', 'New York City', 'New York', '', '1', '0']
['2020-03-03', 'New York City', 'New York', '', '2', '0']
['2020-03-04', 'New York City', 'New York', '', '2', '0']
['2020-03-04', 'Westchester', 'New York', '36119', '9', '0']
['county', 'population']
['Kings', '2559903']
['Queens', '2253858']
['New York', '1628706']
['Suffolk', '1476601']
['Bronx', '1418207']


### 2. Constructing SQL for creating 3 tables in database (one for each of the data files)

In [ ]:
'''
CREATE TABLE IF NOT EXISTS `nycovidcase` (
  `caseid` int(5) NOT NULL AUTO_INCREMENT,
  `date` date,
  `county` varchar(25),
  `state` varchar(25),
  `fips` varchar(10),
  `cases` int(5),
  `death` int(5),
   PRIMARY KEY (`caseid`)
)


CREATE TABLE IF NOT EXISTS `nypopulation` (
  `populationid` int(5) NOT NULL AUTO_INCREMENT,
   `county` varchar(25),
  `population` int(10),
   PRIMARY KEY (`populationid`)
)


)

CREATE TABLE IF NOT EXISTS `nydistance` (
  `distanceid` int(5) NOT NULL AUTO_INCREMENT,
   `fips1` varchar(10),
  `distance` int(4),
  `fips2` varchar(10),
   PRIMARY KEY (`distanceid`)
)
'''

### 3. Connecting database 

In [21]:

import pymysql
conn = pymysql.connect(host='YourDatabaseHostNameOrLocalHost', \
                       port=3306, \
                       user='UserName', \
                       passwd='********', \
                       db='databae_name', autocommit=True)
cur = conn.cursor(pymysql.cursors.DictCursor)


### 4a. Creating Table `nycovidcase`

In [32]:
cur.execute('''

CREATE TABLE IF NOT EXISTS `nycovidcase` (
  `caseid` int(5) NOT NULL AUTO_INCREMENT,
  `date` date,
  
  `county` varchar(25),
  `state` varchar(25),
  `fips` varchar(10),
  `cases` int(5),
  `death` int(5),
   PRIMARY KEY (`caseid`)
);
''')

0

### 4b. Creating Table `nydistance`

In [22]:
cur.execute('''

CREATE TABLE IF NOT EXISTS `nydistance` (
  `distanceid` int(5) NOT NULL AUTO_INCREMENT,
   `fips1` varchar(10),
  `distance` float(4),
  `fips2` varchar(10),
   PRIMARY KEY (`distanceid`)
);
''')

0

### 4c. Creating Table `nypopulation`

In [22]:
# Creating Table nypopulation
cur.execute('''

CREATE TABLE IF NOT EXISTS `nypopulation` (
  `populationid` int(5) NOT NULL AUTO_INCREMENT,
   `county` varchar(25),
  `population` int(10),
   PRIMARY KEY (`populationid`)
);
''')

0

### 5a.  Checking `covid_data_filtered.csv` as a list of dictionary before inserting rows into database table

In [28]:
with open('covid_data_filtered.csv') as f:
    data = [{k: str(v) for k, v in row.items()}
        for row in csv.DictReader(f, skipinitialspace=True)]
    print(data[0])

{'date': '2020-03-01', 'county': 'New York City', 'state': 'New York', 'fips': '', 'cases': '1', 'deaths': '0'} 



### 5b. Inserting rows into `nycovidcase` by utilizing 'block size' concept

Although the csv files here are not too large,still using this concept of sending data in chunks for the sake of a'good practice' so that the codes remain scalable for large data sets 

In [33]:
import pymysql,csv,time,datetime

# preparing 'data' object(a list of dictionary) for inserting rows into the corresponding database table

with open('covid_data_filtered.csv') as f:
    data = [{k: str(v) for k, v in row.items()}
        for row in csv.DictReader(f, skipinitialspace=True)]
    #print(data[0], '\n')

    
# Inserting rows (in blocks, for faster operations) from csv into database table 

sql = '''
INSERT INTO `nycovidcase`
(
`date`, `county`,`state`,`fips`,
`cases`,`death`
)
 VALUES (%s,%s,%s,%s,%s,%s);
 ''' 

tokens =[]
n= 0 
i=0
blocksizes = [4000]

for bs in blocksizes:
    start = time.time()


    for line in data:
        tokens.append((line["date"],
                  line["county"],
                  line["state"],
                  line["fips"],
                  line["cases"],
                  line["deaths"]))
        if i % bs == 0:
            n+=1
            bstart = time.time()
            cur.executemany(sql,tokens)
            conn.commit()
            tokens = []
        i+=1
    print ("block size: " + str(bs) + " - total time : " + str(time.time() - start))
    if len(tokens) > 0:
        cur.executemany(sql,tokens)
        conn.commit()
    
cur.close()

block size: 4000 - total time : 1.097567081451416


### 5c.  Checking `distance_data_filtered.csv` as a list of dictionary before inserting rows into database table

In [23]:
# Checking csv content as a list of dictionary before inserting them into database table
import csv
c=0
with open('distance_data_filtered.csv') as f:
    data = [{k: str(v) for k, v in row.items()}
        for row in csv.DictReader(f, skipinitialspace=True)]
    print(data[0])

{'fips': '36013', 'distance': '359', 'fips2': '36103'} 



### 5d. Inserting rows into `nycdistance`

In [24]:
import pymysql,csv,time,datetime

# preparing 'data' object(a list of dictionary) for inserting rows into the corresponding database table

c=0
with open('distance_data_filtered.csv') as f:
    data = [{k: str(v) for k, v in row.items()}
        for row in csv.DictReader(f, skipinitialspace=True)]
    # print(data[0], '\n')

    
# Inserting rows (in blocks, for faster operations) from csv into database table 
sql = '''
INSERT INTO `nydistance`
(
`fips1`, `distance`,`fips2`
)
 VALUES (%s,%s,%s);
 ''' 

tokens =[]
n= 0 
i=0
blocksizes = [400]

for bs in blocksizes:
    start = time.time()


    for line in data:
        tokens.append((line["fips"],
                  line["distance"],
                  line["fips2"]))
        if i % bs == 0:
            n+=1
            bstart = time.time()
            cur.executemany(sql,tokens)
            conn.commit()
            tokens = []
        i+=1
    print ("block size: " + str(bs) + " - total time : " + str(time.time() - start))
    if len(tokens) > 0:
        cur.executemany(sql,tokens)
        conn.commit()
    
cur.close()

block size: 400 - total time : 0.8525772094726562


### 5e.  Checking `distance_data_filtered.csv` as a list of dictionary before inserting rows into database table

In [3]:
# Checking csv content as a list of dictionary before inserting them into database table
with open('population_data.csv') as f:
    data = [{k: str(v) for k, v in row.items()}
        for row in csv.DictReader(f, skipinitialspace=True)]
    print(data[0:21], '\n')

[{'county': 'Kings', 'population': '2559903'}, {'county': 'Queens', 'population': '2253858'}, {'county': 'New York', 'population': '1628706'}, {'county': 'Suffolk', 'population': '1476601'}, {'county': 'Bronx', 'population': '1418207'}, {'county': 'Nassau', 'population': '1356924'}, {'county': 'Westchester', 'population': '967506'}, {'county': 'Erie', 'population': '918702'}, {'county': 'Monroe', 'population': '741770'}, {'county': 'Richmond', 'population': '476143'}, {'county': 'Onondaga', 'population': '460528'}, {'county': 'Orange', 'population': '384940'}, {'county': 'Rockland', 'population': '325789'}, {'county': 'Albany', 'population': '305506'}, {'county': 'Dutchess', 'population': '294218'}, {'county': 'Saratoga', 'population': '229863'}, {'county': 'Oneida', 'population': '228671'}, {'county': 'Niagara', 'population': '209281'}, {'county': 'Broome', 'population': '190488'}, {'county': 'Ulster', 'population': '177573'}, {'county': 'Rensselaer', 'population': '158714'}] 



### 5f. Inserting rows into `nypopulation`

In [6]:
import pymysql,csv,time,datetime

# preparing 'data' object(a list of dictionary) for inserting rows into the corresponding database table

c=0
with open('population_data.csv') as f:
    data = [{k: str(v) for k, v in row.items()}
        for row in csv.DictReader(f, skipinitialspace=True)]
    # print(data[0], '\n')


# Inserting rows (in blocks, for faster operations) from csv into database table 

sql = '''
INSERT INTO `nypopulation`
(
 `county`,`population`
)
 VALUES (%s,%s)
 ''' 

tokens =[]
n= 0 
i=0
blocksizes = [20]

for bs in blocksizes:
    start = time.time()


    for line in data:
        tokens.append((line["county"],
                  line["population"]))
        if i % bs == 0:
            n+=1
            bstart = time.time()
            cur.executemany(sql,tokens)
            conn.commit()
            tokens = []
        i+=1
    print ("block size: " + str(bs) + " - total time : " + str(time.time() - start))
    if len(tokens) > 0:
        cur.executemany(sql,tokens)
        conn.commit()
    
cur.close()

block size: 20 - total time : 0.2222902774810791
